In [5]:
import pandas as pd
import torch
import random
import os
import re
from torch.distributions.binomial import Binomial
from torch.distributions.bernoulli import Bernoulli
import torch.nn as nn

In [ ]:
def treat_portion(x, pattern):
    score = 0.2
    if re.match(pattern, x) is not None:
        score = 0.8
    return score

DATA_PATH = '/nfs/turbo/lsa-regier/emr-data/'
groups = [str(i) for i in range(10)]
beta_1 = torch.tensor([1.0, 10.0, 100.0])
pattern = 'diag:J45(\d)*,' # Asthma


for group in groups:
    data_path = os.path.join(DATA_PATH, f'group_{group}_merged.csv')
    data_temp = pd.read_csv(data_path, sep=',')
    
    ### calculate propensity score pi(z)
    data_temp = data_temp.assign(base_propensity_score = data_temp['document'].apply(lambda x: treat_portion(x, pattern)))
    
    ### simulate treatment based on propensity score
    all_treatments = Binomial(1, torch.tensor(data_temp["base_propensity_score"].values))
    data_temp = data_temp.assign(treatment = all_treatments.sample().int())

    sigmoid_fn = torch.nn.Sigmoid()
    ### simulate response from treatment and propensity score
    for i in range(2):
        treatment_propensity = 0.25 * i + beta_1[0] * (data_temp['base_propensity_score'] - 0.2)
        input = torch.tensor(treatment_propensity.values)
        reponse_prob = sigmoid_fn(input)
        all_response = torch.bernoulli(reponse_prob)
        data_temp[f'response_{str(i)}_'] = all_response
        
    data_temp['response'] = (data_temp['treatment'] == 0)* data_temp['response_0_'] + (data_temp['treatment'] == 1)* data_temp['response_1_']
    data = pd.concat([data, data_temp]) if group != '0' else data_temp

In [ ]:
data = data.rename(columns={'response_0_': 'potential_response_0', 'response_1_': 'potential_response_1','response_2_': 'response'})

In [ ]:
data["response"].sum()

In [ ]:
data["treatment"].sum()

In [ ]:
SATE = (data["potential_response_1"]-data["potential_response_0"]).mean()
SATE

In [ ]:
data["potential_response_1"].mean()

In [ ]:
data["potential_response_0"].mean()

In [ ]:
data

In [ ]:
((data["treatment"])*data["response"]).sum()

In [ ]:
((1-data["treatment"])*data["response"]).sum()